In [1]:
#test if has specific package from environement
try:
    import pdfquery
    print('package is available from local environement')
except ImportError:
    print('package of local environement is not available')

!pip check

package is available from local environement
No broken requirements found.


In [2]:
#from langchain_community.chat_models import ChatAnthropic, ChatOpenAI
from langchain_openai import ChatOpenAI
from langchain.chains import llm
from langchain_core.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate
)
from langchain.schema import AIMessage, HumanMessage, SystemMessage
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from dotenv import load_dotenv
import PyPDF2
from pdfquery import PDFQuery
import torch



/home/dancinoman/code/dancinoman/legal-explanation/le-package/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
class LegalExpert:
    def __init__(self):



        # falcon model
        model_name = "tiiuae/falcon-11B"
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.falcon_llm = pipeline("text-generation",
                                   model=model_name,
                                   tokenizer=tokenizer,
                                   torch_dtype=torch.float16,
                                   trust_remote_code=True,
                                   device_map="auto")

        print(f'Model {model_name} is set.')
        # create llm pipeline for model
        #model_name = "google/flan-t5-xl"

        #self.huggingface_llm = pipeline("text-generation", model=model_name, tokenizer=tokenizer)
        #print('Hugging face pipeline set.')
        #self.openai_gpt4_llm = ChatOpenAI(temperature=0, max_tokens=256)
        #self.chat = ChatAnthropic()



        #self.chain = llm.LLMChain(llm=self.huggingface_llm, prompt=full_prompt_template)

    def get_system_prompt(self, language, context):
        system_prompt = """
        You are a Canadian Legal Expert.
        Under no circumstances do you give legal advice.

        You are adept at explaining the law in laymans terms, and you are able to provide context to legal questions.
        While you can add context outside of the provided context, please do not add any information that is not directly relevant to the question, or the provided context.
        You speak {language}.
        ### CONTEXT
        {context}
        ### END OF CONTEXT
        """

        return SystemMessagePromptTemplate.from_template(system_prompt)

    def run_chain(self, language, context, question):

        self.system_prompt = self.get_system_prompt(language, context)
        self.user_prompt = HumanMessagePromptTemplate.from_template('{question}')
        
        self.full_prompt_template = ChatPromptTemplate.from_messages(
            [self.system_prompt, self.user_prompt]
        )

        self.chain = self.full_prompt_template | self.falcon_llm

        return self.chain.invoke(input={'context':context, 'language':language,'question':question})




In [4]:
pdf_file_loc = "Legal documentation/Contract_of_PurchaseSale.pdf"

def retrieve_pdf_text(pdf_file_loc):

    pdf_file = PDFQuery(pdf_file_loc)
    pdf_file.load()
    text_elements = pdf_file.pq('LTTextLineHorizontal')
    return "".join([t.text for t in text_elements if t.text.strip() != ''])




In [5]:
# create a streamlit app
print("Starting Document Explainer (that does not give advice)")

machine_reader = LegalExpert()

# create a upload file widget for a pdf


language = input("1.French\n2.English\n")
#question = input("Ask a question? ")
question = 'what is about the document?'

# run the model
legal_response = machine_reader.run_chain(
    language=language, context=retrieve_pdf_text(pdf_file_loc), question=question
)
#Output the answer
print(f"legal_response: {legal_response}")

Starting Document Explainer (that does not give advice)


Loading checkpoint shards: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:15<00:00,  3.17s/it]


Model tiiuae/falcon-11B is set.


1.French
2.English
 2


NotImplementedError: Unsupported message type: <class 'langchain_core.prompts.prompt.PromptTemplate'>